In [62]:
import tensorflow as tf

import numpy as np

import matplotlib.pyplot as plt

import os

import time

from tqdm.notebook import tqdm

In [53]:
cur_dir = os.getcwd()
model_dir_name = "minst_cnn_seq"

ckpt_dir = os.path.join(cur_dir, "checkpoints", model_dir_name)
os.makedirs(ckpt_dir, exist_ok=True)

ckpt_prefix = os.path.join(ckpt_dir, model_dir_name)

In [54]:
mnist = tf.keras.datasets.mnist
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()    

train_images = train_images.astype(np.float32) / 255.
test_images = test_images.astype(np.float32) / 255.

train_images = np.expand_dims(train_images, axis=-1)
test_images = np.expand_dims(test_images, axis=-1)
    
train_labels = tf.keras.utils.to_categorical(train_labels, 10)
test_labels = tf.keras.utils.to_categorical(test_labels, 10)    

batch_size = 100

train_dataset = tf.data.Dataset.from_tensor_slices((train_images, train_labels)).shuffle(len(train_images)).batch(batch_size, drop_remainder=True)
test_dataset = tf.data.Dataset.from_tensor_slices((test_images, test_labels)).batch(batch_size)

In [55]:
# class CreateModel(tf.keras.Model):
#     def __init__(self):
#         super().__init__()
        
#         self.model = tf.keras.Sequential()
    
#         self.model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu, input_shape=(28, 28, 1)))
#         self.model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding="same"))

#         self.model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu))
#         self.model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding="same"))

#         self.model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu))
#         self.model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding="same"))

#         self.model.add(tf.keras.layers.Flatten())

#         self.model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
#         self.model.add(tf.keras.layers.Dropout(0.4))
#         self.model.add(tf.keras.layers.Dense(10))
        
#     def __call__(self, x, training=None, mask=None):
#         y = self.model(x)
#         return y

# network = CreateModel()

# network.model.summary()

In [56]:
def create_model():
    model = tf.keras.Sequential()
    
    model.add(tf.keras.layers.Conv2D(filters=32, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu, input_shape=(28, 28, 1)))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding="same"))
    
    model.add(tf.keras.layers.Conv2D(filters=64, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding="same"))
    
    model.add(tf.keras.layers.Conv2D(filters=128, kernel_size=3, strides=1, padding="same", activation=tf.nn.relu))
    model.add(tf.keras.layers.MaxPool2D(pool_size=2, strides=2, padding="same"))
    
    model.add(tf.keras.layers.Flatten())
    
    model.add(tf.keras.layers.Dense(256, activation=tf.nn.relu))
    model.add(tf.keras.layers.Dropout(0.4))
    model.add(tf.keras.layers.Dense(10))
    
    return model

model = create_model()

model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_21 (Conv2D)           (None, 28, 28, 32)        320       
_________________________________________________________________
max_pooling2d_21 (MaxPooling (None, 14, 14, 32)        0         
_________________________________________________________________
conv2d_22 (Conv2D)           (None, 14, 14, 64)        18496     
_________________________________________________________________
max_pooling2d_22 (MaxPooling (None, 7, 7, 64)          0         
_________________________________________________________________
conv2d_23 (Conv2D)           (None, 7, 7, 128)         73856     
_________________________________________________________________
max_pooling2d_23 (MaxPooling (None, 4, 4, 128)         0         
_________________________________________________________________
flatten_7 (Flatten)          (None, 2048)             

In [57]:
@tf.function
def loss_fn(model, images, labels):
    logits = model(images, training=True)
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, from_logits=True))    
    return loss

@tf.function
def grad_fn(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)


@tf.function
def acc_fn(model, images, labels):
    logits = model(images, training=False)
    is_equal = tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1))
    acc = tf.reduce_mean(tf.cast(is_equal, tf.float32))
    return acc

In [64]:
lr = 0.001
training_epochs = 15

tf.random.set_seed(0)

ckpt = tf.train.Checkpoint(cnn=model)

start_time = time.time()
for epoch in tqdm(range(training_epochs)):
    avg_train_loss = 0.
    avg_train_acc = 0.

    avg_test_acc = 0.
    
    train_step = 0
    test_step = 0    
    
    for images, labels in tqdm(train_dataset):
        grads = grad_fn(model, images, labels)
        optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
        optimizer.apply_gradients(zip(grads, model.variables))
        loss = loss_fn(model, images, labels)
        acc = acc_fn(model, images, labels)
        avg_train_loss += loss
        avg_train_acc += acc
        train_step += 1
        
    avg_train_loss = avg_train_loss / train_step
    avg_train_acc = avg_train_acc / train_step
    
    for images, labels in tqdm(test_dataset):
        acc = acc_fn(model, images, labels)        
        avg_test_acc += acc
        test_step += 1
    
    avg_test_acc = avg_test_acc / test_step    

    print("epoch : {} | loss : {:.6f}| train acc : {:.4f}| test acc :{:.4f}".format(epoch + 1, avg_train_loss, avg_train_acc, avg_test_acc))
    
    ckpt.save(file_prefix=ckpt_prefix)

print("elapsed time :", time.time() - start_time)


epoch : 1 | loss : 0.04229128| train acc : 0.9898| test acc :0.9872



epoch : 2 | loss : 0.02690231| train acc : 0.9931| test acc :0.9899



epoch : 3 | loss : 0.01913621| train acc : 0.9956| test acc :0.9916



epoch : 4 | loss : 0.01473611| train acc : 0.9964| test acc :0.9917



epoch : 5 | loss : 0.01243736| train acc : 0.9972| test acc :0.9934


...


epoch : 12 | loss : 0.00307375| train acc : 0.9993| test acc :0.9907



epoch : 13 | loss : 0.00343869| train acc : 0.9993| test acc :0.9935



epoch : 14 | loss : 0.00234767| train acc : 0.9995| test acc :0.9934



epoch : 15 | loss : 0.00215563| train acc : 0.9995| test acc :0.9933



NameError: name 'start' is not defined